# Problem Statement
- The U.S. has almost 500 students for every guidance counselor. Underserved youth lack the network to find their career role models, making CareerVillage.org the only option for millions of young people in America and around the globe with nowhere else to turn.

- To date, 25,000 volunteers have created profiles and opted in to receive emails when a career question is a good fit for them. This is where your skills come in. To help students get the advice they need, the team at CareerVillage.org needs to be able to send the right questions to the right volunteers. The notifications sent to volunteers seem to have the greatest impact on how many questions are answered.

# Objective: develop a method to recommend relevant questions to the professionals who are most likely to answer them.

In [1]:
import nltk, urllib, request, string, re
from nltk import word_tokenize
from nltk.corpus import stopwords
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
import os
os.chdir('D:\\siuol456\\Documents\\NEU\\Study group 2019 Summer\\FinalProject\\datasets')

In [3]:
Q_body=pd.read_csv('questions.csv')
Q_body.head()

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26 UTC+0000,Teacher career question,What is a maths teacher? what is a ma...
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25 UTC+0000,I want to become an army officer. What can I d...,I am Priyanka from Bangalore . Now am in 10th ...
2,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38 UTC+0000,Will going abroad for your first job increase ...,I'm planning on going abroad for my first job....
3,2f6a9a99d9b24e5baa50d40d0ba50a75,2c30ffba444e40eabb4583b55233a5a4,2017-09-01 14:05:32 UTC+0000,To become a specialist in business management...,i hear business management is a hard way to ge...
4,5af8880460c141dbb02971a1a8369529,aa9eb1a2ab184ebbb00dc01ab663428a,2017-09-01 02:36:54 UTC+0000,Are there any scholarships out there for stude...,I'm trying to find scholarships for first year...


In [47]:
def cleansing(s,n):
    raw = BeautifulSoup(s, 'html.parser').get_text()
    raw=raw.lower()
    raw=re.sub(r'\d+', '', raw)
    tokens = word_tokenize(raw)
    sw = stopwords.words('English')
    new_token=list()
    for t in tokens:
        if t not in sw:
            new_token.append(t)
    punc=list(string.punctuation)
    manualcheck= ['“','”','’',"n't","'m","th","'d","'s","eant",'priyanka','bangalore','std']
    punc.extend(manualcheck)
    new_token2=list()
    for t in new_token:
        if t not in punc:
            new_token2.append(t)
    tagged = nltk.pos_tag(new_token2)
    t=list()
    for com in tagged:
        if com[1]=='NN' or com[1]=='NNS' or com[1]=='NNP' or com[1]=='NNPS':
            t.append(com[0])
    freq= nltk.FreqDist(t)
    tryo = pd.DataFrame(list(dict(freq).items()))
    if tryo.empty:
        result=list()
    else:
        tryo=tryo.sort_values(by=1, ascending=False)
        tryo.columns = ['Words', 'freq']
        if len(list(tryo['Words']))<n:
            result=list(tryo['Words'])
        else:
            df=tryo[0:n]
            result=list(df['Words'])
    return(result)

In [48]:
cleansing(Q_body.questions_body.iloc[1],1)

['college']

In [88]:
Q_body.questions_body[Q_body.questions_id=='72f843e027b24a1ea621b90d479d9db9']

1422    What steps should I take as an undergrad to pr...
Name: questions_body, dtype: object

In [46]:
Q_body.questions_body.iloc[1]

'I am Priyanka from Bangalore . Now am in 10th std . When I go to college I should not get confused on what I want to take to become army officer. So I am asking this question  #military #army'

In [53]:
new_c=list()
for s in Q_body.questions_body:
    w=str(cleansing(s,1))
    new_c.append(w)

In [54]:
new_c[1000:1005]

["['healthcare']", "['design']", "['languages']", "['studies']", "['aid']"]

In [55]:
Q_body['wordlist']=new_c

In [56]:
Q_body.head()

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body,wordlist
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26 UTC+0000,Teacher career question,What is a maths teacher? what is a ma...,['maths']
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25 UTC+0000,I want to become an army officer. What can I d...,I am Priyanka from Bangalore . Now am in 10th ...,['college']
2,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38 UTC+0000,Will going abroad for your first job increase ...,I'm planning on going abroad for my first job....,['job']
3,2f6a9a99d9b24e5baa50d40d0ba50a75,2c30ffba444e40eabb4583b55233a5a4,2017-09-01 14:05:32 UTC+0000,To become a specialist in business management...,i hear business management is a hard way to ge...,['business']
4,5af8880460c141dbb02971a1a8369529,aa9eb1a2ab184ebbb00dc01ab663428a,2017-09-01 02:36:54 UTC+0000,Are there any scholarships out there for stude...,I'm trying to find scholarships for first year...,['scholarships']
